# 🗄️ MBA 564B Module 3 Group Assignment - Part B: SQL

P-Card Transactions Analysis Using SQLite


▶️ First, run the code cell below to import `unittest`, a module used for **🧭 Check Your Work** sections and the autograder.


In [1]:
# DO NOT CHANGE THE CODE IN THIS CELL
import unittest

tc = unittest.TestCase()

▶️ Next, run the code cell below to confirm that the input SQLite database file is present in the same directory as this notebook. If the file is not found, an error message will be displayed.


In [2]:
# DO NOT CHANGE THE CODE IN THIS CELL
import os

# Check if the db file exists in the same directory
if not os.path.exists("denton-pcard-transactions.db"):
    print("Current working directory:", os.getcwd())
    raise Exception(
        "The 'denton-pcard-transactions.db' file is not found in the current directory"
    )

---

### 🎯 Task 1: Import Pandas and NumPy

Import the following Python packages.

- `pandas` using alias `pd`
- `numpy` using alias `np`
- `sqlite3`
- `plotly.express` using alias `px`


In [3]:
# YOUR CODE BEGINS

import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px

# YOUR CODE ENDS

#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [4]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "check-imports-part-b"
_points = 1

tc.assertTrue(
    ("pd" in globals() and hasattr(globals()["pd"], "DataFrame")),
    "Check whether you have correctly import Pandas with the alias pd.",
)
tc.assertTrue(
    "np" in globals() and hasattr(globals()["np"], "ndarray"),
    "Check whether you have correctly import NumPy with the alias np.",
)
tc.assertTrue(
    "sqlite3" in globals() and hasattr(globals()["sqlite3"], "connect"),
    "Check whether you have correctly import sqlite3 module.",
)
tc.assertTrue(
    "px" in globals() and hasattr(globals()["px"], "scatter"),
    "Check whether you have correctly import plotly.express with the alias px.",
)

---

▶️ Run the code cell below to connect to City of Denton's P-Card Transactions SQLite database file `denton-pcard-transactions.db`.

There is only one table in the `denton-pcard-transactions.db` database, named `transactions`. For all tasks in this assignment, you will be querying data from the `transactions` table.


In [5]:
# DO NOT CHANGE THE CODE IN THIS CELL
from pathlib import Path

db_file = "denton-pcard-transactions.db"
db_path = Path(db_file)

with sqlite3.connect(db_path) as conn:
    required_tables = ["transactions"]
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    existing_tables = [row[0] for row in cursor.fetchall()]

    for table in required_tables:
        if table not in existing_tables:
            conn.close()
            del conn
            raise Exception(
                f"❌ The required table '{table}' is missing from the '{db_file}' database. Try deleting and re-downloading the '{db_file}' file."
            )

# if no exception was raised, proceed to connect to the database
conn = sqlite3.connect(db_path)

# df_grading will be used for grading
# df_grading = pd.read_csv(
#     "https://raw.githubusercontent.com/mba564b/canvas-assignment-static-files/refs/heads/main/city-of-denton-pcard/denton-pcard-transactions-2022-2024.csv"
# )

# Writing this code to disable SSL certificate verification which is being blocked by the GitHub URL
import ssl
import io
import urllib.request
_grading_url = "https://raw.githubusercontent.com/mba564b/canvas-assignment-static-files/refs/heads/main/city-of-denton-pcard/denton-pcard-transactions-2022-2024.csv"
_ctx = ssl.create_default_context()
_ctx.check_hostname = False
_ctx.verify_mode = ssl.CERT_NONE
with urllib.request.urlopen(_grading_url, context=_ctx) as _resp:
    df_grading = pd.read_csv(io.BytesIO(_resp.read()))

# display the first 3 rows of the transactions table
display(pd.read_sql_query("SELECT * FROM transactions LIMIT 3;", conn))

,MERCHANT,TRANSACTION_DATE,POST_DATE,CARDHOLDER_NAME,AMOUNT,BUSINESS_UNIT,ACCOUNT
0,GOOGLE,2022-01-01,2022-01-03,Gloria Coughlin,30.00,POLICE SUPPORT,LICENSES
1,TFS,2022-01-01,2022-01-03,Adriana Barker,310.80,WASTEWATER - LABORATORY,SMALL TOOLS & INSTRUMENTS
2,WALMART,2022-01-01,2022-01-03,Jason Eddington,182.54,FIRE OPERATIONS,JANITORIAL SUPPLIES


---

### 🎯 Task 2: Select Transaction and Post Dates from the Database

- Create a SQL query string named `q_select_date_columns`.
- The query should:
  - Select the `TRANSACTION_DATE` column from the
  - Select the `POST_DATE` column
  - Read from the `transactions` table
- Do not include any other columns in the query.


In [6]:
# YOUR CODE BEGINS

q_select_date_columns = """ 
SELECT TRANSACTION_DATE, POST_DATE
FROM transactions
"""

# YOUR CODE ENDS

df_date_columns = pd.read_sql_query(q_select_date_columns, conn)
df_date_columns.head(3)

,TRANSACTION_DATE,POST_DATE
0,2022-01-01,2022-01-03
1,2022-01-01,2022-01-03
2,2022-01-01,2022-01-03


#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [7]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "select-date-columns-sql"
_points = 1

pd.testing.assert_frame_equal(
    df_date_columns.sort_values(by=["TRANSACTION_DATE", "POST_DATE"]).reset_index(
        drop=True
    ),
    df_grading[["TRANSACTION_DATE", "POST_DATE"]]
    .sort_values(by=["TRANSACTION_DATE", "POST_DATE"])
    .reset_index(drop=True),
    check_dtype=False,
)

---

### 🎯 Task 3: Extract year, month, and year-month from transaction date

Using the `transactions` table in the SQLite database, write a SQL query that extracts year, month, and year-month values from the `TRANSACTION_DATE` column.

- Store the SQL query string in a variable named `q_extract_year_month`.
- Use SQLite date functions to extract:
  - The **year** from `TRANSACTION_DATE`
  - The **month** from `TRANSACTION_DATE`
- Create a derived column named **`YEAR_MONTH`** formatted as `"YYYY-MM"` (for example, `"2023-01"`).
- The query output should include the following columns:
  - `TRANSACTION_DATE`
  - `YEAR`
  - `MONTH`: 0-padded month (for example, `01` for January and `12` for December)
  - `YEAR_MONTH`
- Do not perform this task using pandas date methods (such as `.dt`).

**Hint (optional):**
SQLite provides date and time functions such as `strftime()` that can be used to extract parts of a date.

For example, to extract the year from a date column named `TRANSACTION_DATE`, you can use the following SQL snippet:

```sql
SELECT strftime('%Y', TRANSACTION_DATE) AS YEAR FROM transactions;
```

#### 🧾 Expected Output

|     | TRANSACTION_DATE | YEAR | MONTH | YEAR_MONTH |
| --: | :--------------- | ---: | ----: | :--------- |
|   0 | 2022-01-01       | 2022 |    01 | 2022-01    |
|   1 | 2022-01-01       | 2022 |    01 | 2022-01    |


In [9]:
# YOUR CODE BEGINS
q_extract_year_month = """ 
SELECT 
    TRANSACTION_DATE,
    strftime("%Y", TRANSACTION_DATE) AS YEAR,
    strftime("%m", TRANSACTION_DATE) AS MONTH,
    strftime("%Y-%m", TRANSACTION_DATE) AS YEAR_MONTH
FROM 
    transactions
"""

# YOUR CODE ENDS

df_year_month = pd.read_sql_query(q_extract_year_month, conn)
df_year_month.head(2)

,TRANSACTION_DATE,YEAR,MONTH,YEAR_MONTH
0,2022-01-01,2022,01,2022-01
1,2022-01-01,2022,01,2022-01


#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [10]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "extract-year-month-sql"
_points = 2

expected = df_grading[["TRANSACTION_DATE"]].copy()
dt = pd.to_datetime(expected["TRANSACTION_DATE"], errors="coerce")
expected["YEAR"] = dt.dt.strftime("%Y")
expected["MONTH"] = dt.dt.strftime("%m")
expected["YEAR_MONTH"] = dt.dt.strftime("%Y-%m")

pd.testing.assert_frame_equal(
    df_year_month.sort_values(by=["TRANSACTION_DATE"]).reset_index(drop=True),
    expected.sort_values(by=["TRANSACTION_DATE"]).reset_index(drop=True),
    check_dtype=False,
)

---

### 🎯 Task 4: Monthly number of transactions and total amount spent

- Store the SQL query string in a variable named `q_summary_by_year_month`.
- Similar to the previous task, extract the `YEAR_MONTH` from the `TRANSACTION_DATE` column and include it in the output as `YEAR_MONTH`.
- For each `YEAR_MONTH`, calculate:
  - The **total amount spent** (sum of `AMOUNT`)
  - The **number of transactions**
- Name the output columns exactly:
  - `YEAR_MONTH`
  - `AMOUNT` (the summed total)
  - `TRANSACTION_COUNT`
- Sort the output by `YEAR_MONTH` in ascending order.
- Do not perform this task using pandas `groupby`.
- Use SQL aggregation functions instead.

**Hint (optional):**
In SQLite, you can use `GROUP BY`, `SUM()`, and `COUNT()` to aggregate data.

#### 🧾 Expected Output

|     | YEAR_MONTH | AMOUNT | TRANSACTION_COUNT |
| --: | :--------- | -----: | ----------------: |
|   0 | 2022-01    | 257415 |              1133 |
|   1 | 2022-02    | 298910 |              1288 |
|   2 | 2022-03    | 407362 |              1685 |


In [13]:
# YOUR CODE BEGINS

q_summary_by_year_month = """ 
SELECT 
    strftime("%Y-%m", TRANSACTION_DATE) AS YEAR_MONTH,
     SUM(AMOUNT) AS AMOUNT,
     COUNT(*) AS TRANSACTION_COUNT
FROM
    transactions
GROUP BY
    YEAR_MONTH
ORDER By
    YEAR_MONTH ASC
"""

# YOUR CODE ENDS

df_summary_by_year_month = pd.read_sql_query(q_summary_by_year_month, conn)
df_summary_by_year_month.head(3)

,YEAR_MONTH,AMOUNT,TRANSACTION_COUNT
0,2022-01,257415.44,1133
1,2022-02,298910.23,1288
2,2022-03,407361.67,1685


#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [14]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "summary-by-year-month-sql"
_points = 2

df_year_month_grading = df_grading.copy()
df_year_month_grading["YEAR_MONTH"] = pd.to_datetime(
    df_year_month_grading["TRANSACTION_DATE"]
).dt.strftime("%Y-%m")
df_year_month_grading = df_year_month_grading.pivot_table(
    index="YEAR_MONTH", values="AMOUNT", aggfunc=["sum", "count"]
).reset_index()

df_year_month_grading.columns = ["YEAR_MONTH", "AMOUNT", "TRANSACTION_COUNT"]

pd.testing.assert_frame_equal(
    df_summary_by_year_month.reset_index(drop=True),
    df_year_month_grading.reset_index(drop=True),
    check_dtype=False,
    check_exact=False,
)

---

### 🎯 Task 5: Visualize monthly amount spent using a line chart

- Using `df_summary_by_year_month`, create a line chart to visualize the total amount spent ("AMOUNT") for each month ("YEAR_MONTH").
- Use Plotly Express (`px.line()`) to create the line chart.
- Set the x-axis to "YEAR_MONTH" and the y-axis to "AMOUNT".
- Set an appropriate title for the chart using the `title` parameter of `px.line()`.
- Store the figure in a variable named `fig`.

Note that you're not using SQL for this task; instead, you're using the pandas DataFrame `df_summary_by_year_month` created in the previous task.


▶️ Run the code cell below to preview the first 3 rows of the summary dataframe `df_summary_by_year_month` from the previous task.


In [15]:
df_summary_by_year_month.head(3)

,YEAR_MONTH,AMOUNT,TRANSACTION_COUNT
0,2022-01,257415.44,1133
1,2022-02,298910.23,1288
2,2022-03,407361.67,1685


In [16]:
# YOUR CODE BEGINS

fig = px.line(
    df_summary_by_year_month,
    x="YEAR_MONTH",
    y="AMOUNT",
    title="Total P-Card Amount Spent by Month"
)

# YOUR CODE ENDS

fig.show()

#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [17]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "monthly-amount-spent-line-chart"
_points = 1

trace = fig.data[0]
# plotly represents lines as scatter traces with mode containing 'lines'
assert getattr(trace, "type", None) == "scatter", (
    "The trace must be a scatter trace (line chart)."
)
assert list(trace.x) == list(df_summary_by_year_month["YEAR_MONTH"]), (
    "X-axis must be YEAR_MONTH"
)
assert list(trace.y) == list(df_summary_by_year_month["AMOUNT"]), (
    "Y-axis must be AMOUNT"
)
assert fig.layout.title.text is not None and str(fig.layout.title.text).strip() != "", (
    "Chart must include a non-empty title"
)

---

### 🎯 Task 6: Identify transactions that require further review

In order to safeguard municipal funds, Texas requires different bidding requirements at differeing levels of purchases (source: City of Denton 2020 Audit of P-Card Process Report). For purchases over $3,000, the city should attempt to acquire at least three quotes. To help identify large purchases, filter the DataFrame to include only transactions over \$3,000.

- Define a SQL query string named `q_select_over_3000`.
- The query should select all columns from the `transactions` table.
- Include only rows where `AMOUNT` is greater than 3000 (`AMOUNT > 3000`).


In [18]:
q_select_over_3000 = """
SELECT *
FROM transactions
WHERE AMOUNT > 3000;
"""
# YOUR CODE ENDS

df_over_3000 = pd.read_sql_query(q_select_over_3000, conn)
df_over_3000.head(3)

,MERCHANT,TRANSACTION_DATE,POST_DATE,CARDHOLDER_NAME,AMOUNT,BUSINESS_UNIT,ACCOUNT
0,LOWES,2022-01-27,2022-01-28,Brian Noles,4260.0,PM TRADES AND LANDSCAPE,MECHANICAL - ELECTRICAL
1,B,2022-02-21,2022-02-22,Ronda Barnes,3499.0,PROCUREMENT & CMP - PURCHASING,OUTSIDE CONTRACT SERVICES
2,AMAZON,2022-03-30,2022-03-30,Craig Arrington,7717.2,PM TRADES AND LANDSCAPE,MECHANICAL - ELECTRICAL


#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [19]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "filter-transactions-over-3000-sql"
_points = 1

assert (df_over_3000["AMOUNT"] > 3000).all(), (
    "df_over_3000 should only include transactions with AMOUNT > 3000"
)
assert df_over_3000.shape[0] == df_grading[df_grading["AMOUNT"] > 3000].shape[0], (
    "df_over_3000 should only include transactions with AMOUNT > 3000"
)
assert df_over_3000.shape[1] == df_grading[df_grading["AMOUNT"] > 3000].shape[1], (
    "df_over_3000 should include all columns from the original transactions table"
)

---

### 🎯 Task 7: Identify potential split transactions

According to City of Denton's Procurement Policy:

> Unless formally approved through another procurement method, P-Cards may be used only for "Micro Purchases" (goods or services up to $3,000).

Using the `transactions` table in the SQLite database, write a SQL query that identifies potential split transactions. A potential split is defined as multiple transactions by the same cardholder at the same merchant on the same `TRANSACTION_DATE` whose `COMBINED_AMOUNT` exceeds $3,000.

- Store the SQL query string in a variable named `q_split_transactions`.
- Group by `TRANSACTION_DATE`, `CARDHOLDER_NAME`, and `MERCHANT`.
- For each group compute:
  - `COMBINED_AMOUNT` - the sum of `AMOUNT` for the group.
  - `TRANSACTION_COUNT` - the number of transactions in the group.
- Keep only groups with `TRANSACTION_COUNT > 1` **and** `COMBINED_AMOUNT > 3000`.
- Sort results by `COMBINED_AMOUNT` (descending) and then `TRANSACTION_COUNT` (descending).

#### 🧾 Expected Output

Your output should look similar to the table below:

| TRANSACTION_DATE | CARDHOLDER_NAME      | MERCHANT                | COMBINED_AMOUNT | TRANSACTION_COUNT |
| :--------------- | :------------------- | :---------------------- | --------------: | ----------------: |
| 2024-05-03       | Katherine Barnett    | LOWES                   |         34330.0 |                 2 |
| 2024-07-11       | Katherine Barnett    | LOWES                   |         17185.0 |                 3 |
| 2024-08-23       | Carol Martin-Roberts | ATL CENTENNIAL PARK ON  |         11180.7 |                 9 |
| 2022-08-31       | Chaunesy Baker       | ALOFT BY MARRIOTT       |         10357.3 |                 3 |
| 2023-12-12       | Patti Mankin         | USC VITERBI FCCC ONLINE |          9000.0 |                 5 |


In [20]:
# YOUR CODE BEGINS
q_split_transactions = """ 
SELECT 
    TRANSACTION_DATE, CARDHOLDER_NAME, MERCHANT, 
    SUM(AMOUNT) AS COMBINED_AMOUNT,
    COUNT(*) AS TRANSACTION_COUNT
FROM 
    transactions
GROUP By
    TRANSACTION_DATE, CARDHOLDER_NAME, MERCHANT
HAVING 
    COMBINED_AMOUNT > 3000 and TRANSACTION_COUNT > 1
ORDER BY
    COMBINED_AMOUNT DESC, TRANSACTION_COUNT DESC
"""

# YOUR CODE ENDS

df_split_transactions = pd.read_sql_query(q_split_transactions, conn)
df_split_transactions.head()

,TRANSACTION_DATE,CARDHOLDER_NAME,MERCHANT,COMBINED_AMOUNT,TRANSACTION_COUNT
0,2024-05-03,Katherine Barnett,LOWES,34330.0,2
1,2024-07-11,Katherine Barnett,LOWES,17185.0,3
2,2024-08-23,Carol Martin-Roberts,ATL CENTENNIAL PARK ON,11180.7,9
3,2022-08-31,Chaunesy Baker,ALOFT BY MARRIOTT,10357.3,3
4,2023-12-12,Patti Mankin,USC VITERBI FCCC ONLINE,9000.0,5


#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [21]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "potential-split-transactions"
_points = 4

expected = df_grading.pivot_table(
    index=["TRANSACTION_DATE", "CARDHOLDER_NAME", "MERCHANT"],
    values="AMOUNT",
    aggfunc=["sum", "count"],
).reset_index()
expected.columns = [
    "TRANSACTION_DATE",
    "CARDHOLDER_NAME",
    "MERCHANT",
    "COMBINED_AMOUNT",
    "TRANSACTION_COUNT",
]
expected = expected.query("(TRANSACTION_COUNT > 1) and (COMBINED_AMOUNT > 3000)")
expected = expected.sort_values(
    ["COMBINED_AMOUNT", "TRANSACTION_COUNT"], ascending=False
).reset_index(drop=True)

pd.testing.assert_series_equal(
    expected["COMBINED_AMOUNT"].reset_index(drop=True),
    df_split_transactions["COMBINED_AMOUNT"].reset_index(drop=True),
    check_exact=False,
    check_dtype=False,
)

pd.testing.assert_frame_equal(
    expected.sort_values(expected.columns.tolist()).reset_index(drop=True),
    df_split_transactions.sort_values(
        df_split_transactions.columns.tolist()
    ).reset_index(drop=True),
    check_dtype=False,
    check_exact=False,
)

---

### 🧪 Cardholders who have not used their P-Cards in 2023 and 2024

The transaction data spans multiple years (2022–2024). In this task, we want to identify cardholders who made at least one transaction in 2022 but did not make any transactions in 2023 or 2024.

⚠️ **Note**: This task is more advanced than the previous ones. For this reason, the full solution code is provided for you. Read through the SQL query and see if you can understand how it works!


In [22]:
# YOU DO NOT NEED TO CHANGE THE CODE IN THIS CELL
# BUT FEEL FREE TO EXPERIMENT WITH IT TO UNDERSTAND THE SOLUTION
q_inactive_cardholders = """
SELECT DISTINCT CARDHOLDER_NAME
FROM (
    SELECT CARDHOLDER_NAME, CAST(strftime('%Y', TRANSACTION_DATE) AS INTEGER) AS year
    FROM transactions
)
WHERE year = 2022
EXCEPT
SELECT DISTINCT CARDHOLDER_NAME
FROM (
    SELECT CARDHOLDER_NAME, CAST(strftime('%Y', TRANSACTION_DATE) AS INTEGER) AS year
    FROM transactions
)
WHERE year IN (2023, 2024)
ORDER BY CARDHOLDER_NAME;
"""

df_inactive = pd.read_sql_query(q_inactive_cardholders, conn)
inactive_cardholders_2023_2024 = df_inactive["CARDHOLDER_NAME"].tolist()
inactive_cardholders_2023_2024.sort()

# display the first 5 inactive cardholders
display(inactive_cardholders_2023_2024[:5])
print(
    f"Total inactive cardholders in 2023 and 2024: {len(inactive_cardholders_2023_2024)}"
)

['Annie Bunger',
 'Aubrey Shanahan',
 'Billy Downey',
 'Billy Kamp',
 'Bradley Holland']

Total inactive cardholders in 2023 and 2024: 60


---

### 🎯 Task 8: Top 10 merchants by amount spent

- Define a SQL query string named `q_top_10_merchants`.
- The query must compute, for each `MERCHANT`:
  - `TOTAL_AMOUNT` - the sum of `AMOUNT` for that merchant.
  - `TRANSACTION_COUNT` - the number of transactions for that merchant.
- Return exactly three columns (in this order): `MERCHANT`, `TOTAL_AMOUNT`, `TRANSACTION_COUNT`
- Keep only the top 10 merchants by `TOTAL_AMOUNT` (largest totals first) using `LIMIT`.
- Sort the output by `TOTAL_AMOUNT` descending.


In [23]:
# YOUR CODE BEGINS
q_top_10_merchants = """ 
SELECT
    MERCHANT, SUM(AMOUNT) AS TOTAL_AMOUNT, COUNT(*) AS TRANSACTION_COUNT
FROM
    transactions
GROUP BY 
    MERCHANT
ORDER BY
    TOTAL_AMOUNT DESC
LIMIT 10;
"""

# YOUR CODE ENDS

df_top_10_merchants = pd.read_sql_query(q_top_10_merchants, conn)
df_top_10_merchants


,MERCHANT,TOTAL_AMOUNT,TRANSACTION_COUNT
0,AMAZON,2549928.02,21367
1,LOWES,481823.63,2730
2,GRAINGER,371749.00,863
3,THE HOME DEPOT,349217.60,2133
4,INTERNAL,293020.13,491
5,SAM'S CLUB,247918.71,1358
6,AMERICAN,179303.35,542
7,HOME DEPOT,175335.49,1074
8,HILTON,111534.44,238
9,PAYPAL,100802.98,279


#### 🧭 Check Your Work

Run the code cell below to test your solution.

- ✔️ If the code cell runs without errors, you're good to move on.
- ❌ If the code cell produces an error, review your code and fix any mistakes.


In [24]:
# DO NOT CHANGE THE CODE IN THIS CELL
_test_case = "top-10-merchants"
_points = 2

expected = df_grading.pivot_table(
    index="MERCHANT",
    values="AMOUNT",
    aggfunc=["sum", "count"],
).reset_index()
expected.columns = ["MERCHANT", "TOTAL_AMOUNT", "TRANSACTION_COUNT"]
expected = (
    expected.sort_values("TOTAL_AMOUNT", ascending=False)
    .head(10)
    .reset_index(drop=True)
)

pd.testing.assert_frame_equal(
    df_top_10_merchants.reset_index(drop=True),
    expected.reset_index(drop=True),
    check_dtype=False,
    check_exact=False,
)

### 🔌 Close Connection

It's a good practice to close the database connection when you're done to release resources.


▶️ Run the code cell below to close the database connection.


In [25]:
# DO NOT CHANGE THE CODE IN THIS CELL
conn.close()
del conn
print("Connection closed.")

Connection closed.


## 📤 Submission Instructions

There is one final step before exporting the notebook as an .ipynb file for submission. Restart your runtime (kernel) and run all cells from the beginning to ensure that your notebook does not contain any error.

Submit the following files on Canvas:

1. The completed notebook file (.ipynb)
2. HTML-exported version of the notebook (.html)

You do not need to submit the database file (`denton-pcard-transactions.db`).
